In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os
import os
os.chdir('..')

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange


from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#################################################
####### LOAD DLC AND NEURAL ACTIVITY ############
#################################################
from scipy.ndimage import gaussian_filter1d
from matplotlib.gridspec import  GridSpec

#
fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/binarized_traces.npz'
d = np.load(fname,
              allow_pickle=True)

# 
F_thresholded = d['events_F_threshold_upphase']
oasis_smooth = d['events_Oasis_smooth_upphase']
spikes= d['spikes']

# 
print ("Cell spikes: ", spikes.shape)
print ("Cell F: ", F_thresholded.shape)

#
locs = np.load('/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/locs.npy')
print (locs.shape)
plt.scatter(locs[:, 0], locs[:, 1],
           alpha=.4)
plt.show()

Cell spikes:  (203, 36026)
Cell F:  (203, 36026)
(36000, 2)


In [4]:
###########################################################
################# GENERATE CELL MAPS ######################
###########################################################
from tqdm import tqdm

# make gaussian kernel
width=300
x, y = np.meshgrid(np.linspace(-1,1,width), np.linspace(-1,1,width))
dst = np.sqrt(x*x+y*y)
sigma = .2
muu = 0.000  
scaler = 10
gauss = np.exp(-( (dst-muu)**2 / ( scaler * sigma**2 ) ) )



###############################################################
###############################################################
###############################################################
fig=plt.figure()
units=np.arange(0,100,1)
scaled=True
ctr=0
imgs = []
for unit in tqdm(units):
    ax=plt.subplot(10,10,ctr+1)
    #ax=plt.subplot(2,5,ctr+1)
    spks = oasis_smooth[unit]
    spks = spikes[unit]
    idx = np.where(spks>0)[0]
    
    if False:
        #idx += np.random.randint(-100,1E3,idx.shape[0])
        idx += 5000

    # 
    img = np.zeros((2000,1600),'float32')
    for id_ in idx:
        #
        try:
            temp_loc = np.int32(locs[id_])
        except:
            continue

        #
        scale=1.0
        if scaled:
            scale=spks[id_]

        try:
            img[temp_loc[0]-width//2:temp_loc[0]+width//2,
                temp_loc[1]-width//2:temp_loc[1]+width//2]+= gauss*scale
        except:
            pass

    img = img[350:1800,100:1400]
    plt.ylabel(str(unit))
    plt.imshow(img,
              vmin=0,
              vmax=np.max(img),
              cmap='jet')
    plt.xticks([])
    plt.yticks([])
    imgs.append(img)
    
    ctr+=1
plt.show()



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:54<00:00,  1.82it/s]


In [13]:
##################################################################
##################################################################
##################################################################
from tqdm import tqdm

# make gaussian kernel
width=200
x, y = np.meshgrid(np.linspace(-1,1,width), 
                   np.linspace(-1,1,width))

# 
dst = np.sqrt(x*x+y*y)
sigma = .2
muu = 0.001
scaler = 10
gauss = np.exp(-( (dst-muu)**2 / ( scaler * sigma**2 ) ) )

fig=plt.figure()
# units=np.arange(100)
units = [31,39,97]
ctr=0
labels=['F_upphase','Oasis_smooth','Oasis spikes']
scaled=False
for unit in tqdm(units):

    # 
    for p in range(3):
        ax=plt.subplot(3,len(units),ctr+1+p*3)
        if p==0:
            spks = F_thresholded[unit]
        elif p==1:
            spks = oasis_smooth[unit]
        elif p==2:
            spks = spikes[unit]
            
        if ctr==0:
            plt.ylabel(labels[p])
        
        # 
        idx = np.where(spks>0)[0]
        #idx = np.arange(spks.shape[0])

        if False:
            #idx += np.random.randint(-100,1E3,idx.shape[0])
            idx += 5000

        # 
        img = np.zeros((2000,1600),'float32')
        for id_ in idx:

            #
            try:
                temp_loc = np.int32(locs[id_])
            except:
                continue

            #
            scale=1.0
            if scaled:
                scale=spks[id_]
                
            try:
                img[temp_loc[0]-width//2:temp_loc[0]+width//2,
                    temp_loc[1]-width//2:temp_loc[1]+width//2]+= gauss*scale
            except:
                pass

        img = img[500:1700,200:1300]
        plt.title("cell : "+ str(unit))
        plt.imshow(img,
                  vmin=0,
                  vmax=np.max(img),
                  cmap='jet')
        plt.xticks([])
        plt.yticks([])
        
    ctr+=1
plt.show()








100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.75it/s]
